# 4.4.1 Phân cụm khách hàng dựa trên Tổng chi tiêu (Montage) và tần suất mua hàng (Frequency)

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from datetime import datetime
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris

## Tiền xử lý dữ liệu và chuẩn hóa dữ liệu

In [ ]:
# 1. Đọc dữ liệu từ Iris dataset
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)



print(df)
# # Loại bỏ các hóa đơn hủy (các giá trị InvoiceNo bắt đầu bằng 'C')
# df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]


# # Tính toán cột Total: Số lượng * Đơn giá
# df['Total'] = df['Quantity'] * df['UnitPrice']


# # 2. Làm sạch dữ liệu

# # Loại bỏ các hóa đơn hủy (InvoiceNo bắt đầu bằng 'C')
# df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Loại bỏ giá trị null
df = df.dropna()
    
# Loại bỏ giá trị trùng lặp
df = df.drop_duplicates()

# # Loại bỏ các hàng có Quantity < 0
# df = df[df["Quantity"] >= 0]

# # Chuyển đổi CustomerID thành kiểu số nguyên để dễ xử lý
# df['CustomerID'] = df['CustomerID'].astype(int)

# # Tính toán cột "Total" = Số lượng * Đơn giá (Monetary Value cho mỗi giao dịch)
# df['Total'] = df['Quantity'] * df['UnitPrice']

# # 3. Tổng hợp dữ liệu theo từng khách hàng
# customer_agg = df.groupby('CustomerID').agg({
#     'InvoiceNo': 'nunique',   # Tần suất giao dịch (số hóa đơn duy nhất) → Frequency
#     'Total': 'sum',           # Tổng số tiền chi tiêu của khách hàng → Monetary
#     'InvoiceDate': 'max'      # Ngày giao dịch gần nhất → để tính Recency
# }).reset_index()  # Reset index để tránh MultiIndex

# # Đổi tên cột cho dễ hiểu
# customer_agg.rename(columns={'InvoiceNo': 'Frequency', 'Total': 'Monetary'}, inplace=True)

# # 4. Tính "Recency" (Số ngày kể từ lần mua gần nhất)
# customer_agg['InvoiceDate'] = pd.to_datetime(customer_agg['InvoiceDate'])  # Chuyển về kiểu datetime
# reference_date = customer_agg['InvoiceDate'].max()  # Ngày mới nhất trong dữ liệu
# customer_agg['Recency'] = (reference_date - customer_agg['InvoiceDate']).dt.days  # Tính số ngày từ lần mua cuối

# # 5. Chọn các đặc trưng cần thiết để phân cụm
# X = customer_agg[['Monetary', 'Frequency', 'Recency']]

# # 6. Chuẩn hóa dữ liệu bằng StandardScaler (giúp mô hình phân cụm hoạt động tốt hơn)
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)  # Chuẩn hóa dữ liệu để đưa về phân phối chuẩn (mean = 0, std = 1)

# print(customer_agg.info())
# print(customer_agg.head())




     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]
